In [1]:
import cobra.io
import escher
import matplotlib.pyplot as plt
from escher import Builder
from cobra import Model, Reaction, Metabolite
import numpy as np
import pandas as pd
import math

## Inspect Growth metabolism

In [4]:
pth = cobra.io.read_sbml_model('../../Models/p-thermo_anaerob_actn.xml')
pth.reactions.GAPDH.bounds = 0,1000 #Futile cycle discovered, due to error in bounds

medium = pth.medium
medium["EX_glc__D_e"] = 0
medium["EX_o2_e"] = 1000
medium["EX_ac_e"] = 10
pth.medium = medium

OSError: [Errno The file with 'filename' does not exist, or is not an SBML string. Provide the path to an existing SBML file or a valid SBML string representation: 
%s] ../Models/p-thermo_anaerob_actn.xml

In [ ]:
Growth = pth

Growth.reactions.ATPM.bounds = 0,1000 #Ignore NGAM

solution = cobra.flux_analysis.pfba(Growth)
print(solution.fluxes.ATPM)
Growth.summary()

In [ ]:
Builder(model = Growth, map_json = '../../Models/EscherMaps/P-thermo_AcetateDegrad.json', reaction_data = solution.fluxes.to_dict())

## Inspect Acetone production metabolism

In [ ]:
Acetn  = pth

Acetn.reactions.PTAr.bounds = -1000,1000
Acetn.reactions.ACKr.bounds = -1000,1000

Acetn.objective = "EX_act_e" #don't force growth

Acetn.reactions.ATPS4r.bounds = 0,0
Acetn.reactions.ATPM.bounds = -1000,1000 #Ignore NGAM

solution = cobra.flux_analysis.pfba(Acetn)
print(solution.fluxes.ATPM)
Acetn.summary()

In [ ]:
Builder(model = Acetn, map_json = '../../Models/EscherMaps/P-thermo_AcetateDegrad.json', reaction_data = solution.fluxes.to_dict())

In [ ]:
Builder(model = Acetn, map_json = '../../Models/EscherMaps/AcetonePathway.json', reaction_data = solution.fluxes.to_dict())

## Inspect Respiration Metabolism

In [ ]:
Resp = cobra.io.read_sbml_model('../../Models/p-thermo_anaerob_actn.xml')

Resp.reactions.GAPDH.bounds = 0,1000 #Futile cycle discovered, due to error in bounds

medium = Resp.medium
medium["EX_glc__D_e"] = 0
medium["EX_o2_e"] = 1000
medium["EX_ac_e"] = 1
Resp.medium = medium

Resp.objective = "ATPM" #don't force growth

Resp.reactions.ATPM.bounds = 0,1000 #Ignore NGAM

solution = cobra.flux_analysis.pfba(Resp)
Resp.summary()

In [ ]:
Builder(model = Resp, map_json = '../../Models/EscherMaps/P-thermo_AcetateDegrad.json', reaction_data = solution.fluxes.to_dict())